# Two Core Agent Patterns

All agentic frameworks (LangChain, LangGraph, CrewAI, DSPy) use these **two patterns**:

1. **Tool Binding**: LLM decides which tools to call
2. **Structured Output**: LLM returns predefined schema

This notebook shows both patterns in action.

## Setup

In [ ]:
!pip install -q langchain langchain-openai

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env file

openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Pattern 1: Tool Binding

**Use case**: LLM needs to perform actions (search, calculate, call APIs)

**Flow**:
1. Define tools with `@tool` decorator
2. Bind tools to LLM: `llm.bind_tools(tools)`
3. LLM suggests which tools to call
4. You execute the tools

In [2]:
# Step 1: Define tools
@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
  """Subtract two numbers."""
  return a - b

@tool
def divide(a: float, b: float) -> float:
  """Divide two numbers."""
  return a / b

tools = [multiply, add, subtract, divide]

In [4]:
# Step 2: Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Step 3: LLM decides which tool to use
response = llm_with_tools.invoke("What is 25 times 4?")

print("Tool Calls:", response.tool_calls)

Tool Calls: [{'name': 'multiply', 'args': {'a': 25, 'b': 4}, 'id': 'call_2l77LxQb9YsEGoRTmxlVCr8k', 'type': 'tool_call'}]


In [5]:
# Step 4: Execute the tool
tool_call = response.tool_calls[0]
tool_name = tool_call['name']
tool_args = tool_call['args']

# Find and run the tool
tool_map = {t.name: t for t in tools}
result = tool_map[tool_name].invoke(tool_args)

print(f"\nResult: {result}")


Result: 100.0


**Key insight**: LLM only *suggests* tool calls. You must execute them.

## Pattern 2: Structured Output

**Use case**: Extract/parse data into a specific format

**Flow**:
1. Define Pydantic model (schema)
2. Use `llm.with_structured_output(Model)`
3. LLM returns validated typed objects

In [6]:
# Step 1: Define schema
class Person(BaseModel):
    name: str = Field(description="Person's full name")
    email: str = Field(description="Email address")
    age: int = Field(description="Age in years")

In [7]:
# Step 2: Create structured LLM
extractor = llm.with_structured_output(Person)

# Step 3: Extract data
text = "Hi, I'm Alice Johnson, 28 years old. Email: alice@example.com"
person = extractor.invoke(text)

print(f"Type: {type(person)}")
print(f"Name: {person.name}")
print(f"Email: {person.email}")
print(f"Age: {person.age}")

Type: <class '__main__.Person'>
Name: Alice Johnson
Email: alice@example.com
Age: 28


**Key insight**: You get typed Python objects, not strings or dicts!

## Comparison

| Pattern | Method | When to Use | Output |
|---------|--------|-------------|--------|
| Tool Binding | `.bind_tools()` | Need to perform actions | Tool call suggestions |
| Structured Output | `.with_structured_output()` | Need to extract/parse data | Typed Python objects |

### Examples

**Use Tool Binding for:**
- Search the web
- Call APIs
- Execute code
- File operations

**Use Structured Output for:**
- Extract entities
- Parse forms
- Generate task lists
- Classify text

## Framework Comparison

All frameworks use the two patterns above, but with different approaches:

### LangGraph
**What**: Low-level framework for building stateful multi-agent workflows

**Pros:**
- Full control over agent logic and state
- Built-in persistence and checkpointing
- Great for complex workflows with cycles and conditions
- Explicit graph structure makes debugging easier

**Cons:**
- Steeper learning curve
- More boilerplate code
- Need to manually handle state management

**When to use:** Complex multi-step workflows, need full control, production systems

---

### DSPy
**What**: Optimization-focused framework that compiles prompts

**Pros:**
- Automatically optimizes prompts and tool use
- Focus on what to do, not how (declarative)
- Great for research and experimentation
- Can improve performance without manual prompt engineering

**Cons:**
- Different paradigm (requires mindset shift)
- Less mature ecosystem
- Optimization can be slow
- Less intuitive for traditional developers

**When to use:** Research projects, need to optimize performance, willing to learn new paradigm

---

### CrewAI
**What**: High-level framework with role-based agents

**Pros:**
- Very simple API, minimal boilerplate
- Built-in agent roles and collaboration
- Good for team-based workflows
- Fast prototyping

**Cons:**
- Less control over internals
- Opinionated architecture
- Harder to debug complex issues
- Limited customization

**When to use:** Quick prototypes, team-based agents, don't need fine-grained control

---

### LangFlow
**What**: Visual/no-code builder for LangChain workflows

**Pros:**
- Visual drag-and-drop interface
- No coding required for basic flows
- Great for demos and exploration
- Easy to share workflows

**Cons:**
- Limited to pre-built components
- Hard to version control
- Not suitable for complex logic
- Performance overhead

**When to use:** Demos, non-developers, rapid experimentation, visual learners

---

## Quick Decision Guide

```
Need full control + complex logic? → LangGraph
Want to optimize performance? → DSPy
Need quick prototype with teams? → CrewAI
Building demo or learning? → LangFlow
Just starting out? → LangChain (plain)
```

**Recommendation**: Start with plain LangChain to learn the patterns, then graduate to LangGraph for production.